In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
)
from theme import apply_theme
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

In [ ]:
WAVE_COLOR = [
    '#D45E00', # '#BA4338', # early
    '#0072B2', # late
    'black'
]
STROKE = None
AXIS_SHOW = alt.Axis(grid=True, labels=True, ticks=True, domain=True, tickMinStep=1)
AXIS_HIDE = alt.Axis(grid=True, labels=False, ticks=False, domain=True)

In [ ]:
df = pd.read_csv(join("..", "data", "1.1.resurgence", "demographics", "demographic_stats.csv"))

# List of values to get a better sense on the data
print(
    # Singapore data shouldn't be included here
    # demo_df.country.unique().tolist(),
    df.group.unique().tolist(),
    # demo_df.country_wave.unique().tolist()
) 

df

In [ ]:
ageGroups = ['0-25', '26-49', '50-69', '70-79', '80+']
sexGroups = ['Female', 'Male']
raceGroups = ['White', 'Black']

#### Rename columns and values so that it looks more organized

In [ ]:
cdf = df.copy()
cdf = cdf.rename(columns={
    'p.all': 'p_all', 
    'n.all': 'n_all'
})
cdf.group = df.group.apply(
    lambda x: {
        '00to25': '0-25',
        '26to49': '26-49',
        '50to69': '50-69',
        '70to79': '70-79',
        '80plus': '80+',
        'female': 'Female',
        'male': 'Male',
        'white': 'White',
        'black': 'Black',
        'other': 'Other'
    }[x]
)
cdf.wave = df.wave.apply(
    lambda x: {
        'early': 'Early',
        'late': 'Late'
    }[x]
)
cdf

#### Age

In [ ]:
agedf = cdf[cdf.group.isin(ageGroups)]
agedf

In [ ]:
"""
Age percentage bar chart
"""
agebar_base = alt.Chart(
    agedf
).encode(
    x=alt.X(
        'group:N', 
        title=None, 
        axis=AXIS_HIDE
    ),
    y=alt.Y('p_all:Q', axis=alt.Axis(format='.0%'), title="Percentage of Patients"),
    color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late', 'Late - Early'], range=WAVE_COLOR), title="Wave")
).properties(
#     title={
#         "text": 'Age Groups by Wave',
#         "dx": 0,
#         "subtitle": get_visualization_subtitle(data_release='2021-01-25', num_sites='?'), 
#         "subtitleColor": "gray",
#     },
    width=280, height=200
)

agebar_early = agebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Early']}
).mark_bar(
    xOffset=-10,
    size=20, stroke=STROKE
)
agebar_late = agebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Late']}
).mark_bar(
    xOffset=10,
    size=20, stroke=STROKE
)

agebar = alt.layer(agebar_early, agebar_late)

"""
Age N size bar chart
"""
nagebar_base = alt.Chart(
    agedf
).encode(
    x=alt.X(
        'group:N', 
        title='Age', 
        axis=AXIS_SHOW
    ),
    y=alt.Y('n_all:Q', title="# of Patients"),
    color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave")
).properties(
    width=280, height=70
)

nagebar_early = nagebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Early']}
).mark_bar(
    xOffset=-10,
    size=20, stroke=STROKE
)
nagebar_late = nagebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Late']}
).mark_bar(
    xOffset=10,
    size=20, stroke=STROKE
)
nagebar = alt.layer(nagebar_early, nagebar_late)

# agebar = alt.vconcat(agebar, nagebar)

# agebar = apply_theme(
#     agebar,
#     legend_orient='top-left'
# )

# rule = agebar_base.mark_rule(color='grey').encode(
#     x='group:N',
#     size=alt.value(10)
# )

(agebar)

#### Sex

In [ ]:
sexdf = cdf[cdf.group.isin(sexGroups)]
sexdf

In [ ]:
"""
Visual parameters
"""
width = 140

"""
Sex percentage bar chart
"""
sexbar_base = alt.Chart(
    sexdf
).encode(
    x=alt.X(
        'group:N', 
        title=None, 
        axis=AXIS_HIDE
    ),
    y=alt.Y('p_all:Q', axis=alt.Axis(format='.0%', grid=True, labels=False, ticks=False, domain=False), title=None),
    color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave")
).properties(
#     title={
#         "text": 'Sex Group by Wave',
#         "dx": 0,
#         "subtitle": get_visualization_subtitle(data_release='2021-01-25', num_sites='?'), 
#         "subtitleColor": "gray",
#     },
    width=width, height=200
)

sexbar_early = sexbar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Early']}
).mark_bar(
    xOffset=-10,
    size=20, stroke=STROKE
)
sexbar_late = sexbar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Late']}
).mark_bar(
    xOffset=10,
    size=20, stroke=STROKE
)

sexbar = alt.layer(sexbar_early, sexbar_late)

"""
Sex N size bar chart
"""
nsexbar_base = alt.Chart(
    sexdf
).encode(
    x=alt.X(
        'group:N', 
        title='Sex', 
        axis=AXIS_SHOW
    ),
    y=alt.Y('n_all:Q', axis=alt.Axis(title=None, grid=True, labels=False, ticks=False, domain=False), title=None),
    color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave")
).properties(
    width=width, height=70
)

nsexbar_early = nsexbar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Early']}
).mark_bar(
    xOffset=-10,
    size=20, stroke=STROKE
)
nsexbar_late = nsexbar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Late']}
).mark_bar(
    xOffset=10,
    size=20, stroke=STROKE
)
nsexbar = alt.layer(nsexbar_early, nsexbar_late)

# sexbar = alt.vconcat(sexbar, nsexbar)

# sexbar = apply_theme(
#     sexbar,
#     legend_orient='top-left'
# )

sexbar

#### Race

In [ ]:
racedf = cdf[cdf.group.isin(raceGroups)]
racedf

In [ ]:
"""
Visual parameters
"""
width = 140

"""
Race percentage bar chart
"""
racebar_base = alt.Chart(
    racedf
).encode(
    x=alt.X(
        'group:N', 
        title=None, 
        axis=AXIS_HIDE
    ),
    y=alt.Y('p_all:Q', axis=alt.Axis(format='.0%', grid=True, labels=False, ticks=False, domain=False), title=None),
    color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave", legend=alt.Legend(title="Wave"))
).properties(
#     title={
#         "text": 'Sex Group by Wave',
#         "dx": 0,
#         "subtitle": get_visualization_subtitle(data_release='2021-01-25', num_sites='?'), 
#         "subtitleColor": "gray",
#     },
    width=width, height=200
)

racebar_early = racebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Early']}
).mark_bar(
    xOffset=-10,
    size=20, stroke=STROKE
)
racebar_late = racebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Late']}
).mark_bar(
    xOffset=10,
    size=20, stroke=STROKE
)

racebar = alt.layer(racebar_early, racebar_late)

"""
Race N size bar chart
"""
nracebar_base = alt.Chart(
    racedf
).encode(
    x=alt.X(
        'group:N', 
        title='Race', 
        axis=AXIS_SHOW
    ),
    y=alt.Y('n_all:Q', axis=alt.Axis(title=None, grid=True, labels=False, ticks=False, domain=False), title=None),
    color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave")
).properties(
    width=width, height=70
)

nracebar_early = nracebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Early']}
).mark_bar(
    xOffset=-10,
    size=20, stroke=STROKE
)
nracebar_late = nracebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Late']}
).mark_bar(
    xOffset=10,
    size=20, stroke=STROKE
)
nracebar = alt.layer(nracebar_early, nracebar_late)

# sexbar = alt.vconcat(sexbar, nsexbar)

# sexbar = apply_theme(
#     sexbar,
#     legend_orient='top-left'
# )

racebar

In [ ]:
final_chart = alt.hconcat(agebar, sexbar, racebar).resolve_scale(y='shared', color='shared')

nfinal_chart = alt.hconcat(nagebar, nsexbar, nracebar).resolve_scale(y='shared', color='shared')

final_chart = (
    (final_chart & nfinal_chart).properties(
        title={
            "text": 'Demographics by Wave',
            "dx": 80,
            "subtitle": get_visualization_subtitle(data_release='2021-01-25', with_num_sites=False), 
            "subtitleColor": "gray",
        }
    )
)

final_chart = apply_theme(
    final_chart,
    title_anchor='start',
    legend_orient='top-left'
)
final_chart

#### Add Diff Ticks

In [ ]:
diff = cdf.copy()
diff.loc[diff.wave == 'Early', 'p_all'] = diff[diff.wave == 'Early'].p_all.apply(lambda x: -x)
diff = diff.groupby(['group']).sum()
diff = diff.reset_index()

"""
VISUAL PARAMETERS
"""
xOffset = -7

"""
AGE DIFF
"""
age_diff = diff[diff.group.isin(ageGroups)]

age_diff_chart = alt.Chart(
    age_diff
).mark_tick(
    color="black", size=40, stroke="white", thickness=5, xOffset=xOffset
).encode(
    x=alt.X("group:N", title=None, axis=None),
    y=alt.Y(
        "p_all:Q", 
#         title=None, 
#         axis=alt.Axis(format=".0%", grid=True, labels=True, ticks=False, domain=False)
    )
)

"""
AGE DIFF
"""
sex_diff = diff[diff.group.isin(sexGroups)]

sex_diff_chart = alt.Chart(
    sex_diff
).mark_tick(
    color="black", size=40, stroke="white", thickness=5, xOffset=xOffset
).encode(
    x=alt.X("group:N", title=None, axis=None),
    y=alt.Y(
        "p_all:Q",
    )
)

"""
AGE DIFF
"""
race_diff = diff[diff.group.isin(raceGroups)]

race_diff_chart = alt.Chart(
    race_diff
).mark_tick(
    color="black", size=40, stroke="white", thickness=5, xOffset=xOffset
).encode(
    x=alt.X("group:N", title=None, axis=None),
    y=alt.Y(
        "p_all:Q",
    )
)

diff
diffchart
# cdf
# age_diff

In [ ]:
final_chart = alt.hconcat(
    (agebar + age_diff_chart), 
    (sexbar + sex_diff_chart), 
    (racebar + race_diff_chart)
).resolve_scale(y='shared', color='shared')

nfinal_chart = alt.hconcat(nagebar, nsexbar, nracebar).resolve_scale(y='shared', color='shared')

final_chart = (
    (final_chart & nfinal_chart).properties(
        title={
            "text": 'Demographics by Wave',
            "dx": 80,
            "subtitle": get_visualization_subtitle(data_release='2021-01-25', with_num_sites=False), 
            "subtitleColor": "gray",
        }
    )
)

final_chart = apply_theme(
    final_chart,
    title_anchor='start',
    legend_orient='right'
)
final_chart

# Country-level

In [ ]:
df = pd.read_csv(join("..", "data", "1.1.resurgence", "demographics", "demographic_stat_bycountry.csv"))

# List of values to get a better sense on the data
print(
    # Singapore data shouldn't be included here
    # demo_df.country.unique().tolist(),
    df.group.unique().tolist(),
    # demo_df.country_wave.unique().tolist()
) 

df

In [ ]:
def FUNC_DEMOGRAPHICS_BY_WAVE(_data, title='Demographics by Wave', race=True):
    d = _data.copy()
    
    """
    RENAME COLUMNS AND VALUES
    """
    d = d.rename(columns={
        'p.all': 'p_all', 
        'n.all': 'n_all'
    })
    d.group = d.group.apply(
        lambda x: {
            '00to25': '0-25',
            '26to49': '26-49',
            '50to69': '50-69',
            '70to79': '70-79',
            '80plus': '80+',
            'female': 'Female',
            'male': 'Male',
            'white': 'White',
            'black': 'Black',
            'other': 'Other',
            'other_age': 'Other',
            'other_sex': 'Other',
            'other_race': 'Other'
        }[x]
    )
    d.wave = d.wave.apply(
        lambda x: {
            'early': 'Early',
            'late': 'Late'
        }[x]
    )
    
    """
    CATEGORIES WE USE
    """
    AGE_GROUPS = ['0-25', '26-49', '50-69', '70-79', '80+']
    SEX_GROUPS = ['Female', 'Male']
    RACE_GROUPS = ['White', 'Black']
    
    """
    /////////////////////
    SUB-CHARTS FOR GROUPS
    /////////////////////
    """
    
    """
    COMMON VISUAL PARAMETERS
    """
    width = 140
    
    """
    AGE GROUPS
    """
    ad = d[d.group.isin(AGE_GROUPS)]
    
    ############## Bar Chart for % of Participants ##############
    age_p_bar_base = alt.Chart(
        ad
    ).encode(
        x=alt.X(
            'group:N', 
            title=None, 
            axis=AXIS_HIDE
        ),
        y=alt.Y('p_all:Q', axis=alt.Axis(format='.0%'), title="Percentage of Patients"),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late', 'Late - Early'], range=WAVE_COLOR), title="Wave")
    ).properties(
        width=280, height=200
    )

    age_p_bar_early = age_p_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Early']}
    ).mark_bar(
        xOffset=-10,
        size=20, stroke=STROKE
    )
    age_p_bar_late = age_p_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Late']}
    ).mark_bar(
        xOffset=10,
        size=20, stroke=STROKE
    )

    age_p_bar = alt.layer(age_p_bar_early, age_p_bar_late)

    ############## Bar Chart for # of Participants ##############
    age_n_bar_base = alt.Chart(
        ad
    ).encode(
        x=alt.X(
            'group:N', 
            title='Age', 
            axis=AXIS_SHOW
        ),
        y=alt.Y('n_all:Q', title="# of Patients"),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave")
    ).properties(
        width=280, height=70
    )

    age_n_bar_early = age_n_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Early']}
    ).mark_bar(
        xOffset=-10,
        size=20, stroke=STROKE
    )
    age_n_bar_late = age_n_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Late']}
    ).mark_bar(
        xOffset=10,
        size=20, stroke=STROKE
    )
    
    age_n_bar = alt.layer(age_n_bar_early, age_n_bar_late)
    
    """
    SEX GROUPS
    """    
    sd = d[d.group.isin(SEX_GROUPS)]

    ############## Bar Chart for % of Participants ##############
    sex_p_bar_base = alt.Chart(
        sd
    ).encode(
        x=alt.X(
            'group:N', 
            title=None, 
            axis=AXIS_HIDE
        ),
        y=alt.Y('p_all:Q', axis=alt.Axis(format='.0%', grid=True, labels=False, ticks=False, domain=False), title=None),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave")
    ).properties(
        width=width, height=200
    )

    sex_p_bar_early = sex_p_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Early']}
    ).mark_bar(
        xOffset=-10,
        size=20, stroke=STROKE
    )
    sex_p_bar_late = sex_p_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Late']}
    ).mark_bar(
        xOffset=10,
        size=20, stroke=STROKE
    )

    sex_p_bar = alt.layer(sex_p_bar_early, sex_p_bar_late)

    ############## Bar Chart for # of Participants ##############
    sex_n_bar_base = alt.Chart(
        sd
    ).encode(
        x=alt.X(
            'group:N', 
            title='Sex', 
            axis=AXIS_SHOW
        ),
        y=alt.Y('n_all:Q', axis=alt.Axis(title=None, grid=True, labels=False, ticks=False, domain=False), title=None),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave")
    ).properties(
        width=width, height=70
    )

    sex_n_bar_early = sex_n_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Early']}
    ).mark_bar(
        xOffset=-10,
        size=20, stroke=STROKE
    )
    sex_n_bar_late = sex_n_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Late']}
    ).mark_bar(
        xOffset=10,
        size=20, stroke=STROKE
    )
    sex_n_bar = alt.layer(sex_n_bar_early, sex_n_bar_late)
    
    """
    RACE GROUPS
    """    
    rd = d[d.group.isin(RACE_GROUPS)]

    ############## Bar Chart for % of Participants ##############
    race_p_bar_base = alt.Chart(
        rd
    ).encode(
        x=alt.X(
            'group:N', 
            title=None, 
            axis=AXIS_HIDE
        ),
        y=alt.Y('p_all:Q', axis=alt.Axis(format='.0%', grid=True, labels=False, ticks=False, domain=False), title=None),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave", legend=alt.Legend(title="Wave"))
    ).properties(
        width=width, height=200
    )

    race_p_bar_early = race_p_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Early']}
    ).mark_bar(
        xOffset=-10,
        size=20, stroke=STROKE
    )
    race_p_bar_late = race_p_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Late']}
    ).mark_bar(
        xOffset=10,
        size=20, stroke=STROKE
    )

    race_p_bar = alt.layer(race_p_bar_early, race_p_bar_late)

    ############## Bar Chart for # of Participants ##############
    race_n_bar_base = alt.Chart(
        rd
    ).encode(
        x=alt.X(
            'group:N', 
            title='Race', 
            axis=AXIS_SHOW
        ),
        y=alt.Y('n_all:Q', axis=alt.Axis(title=None, grid=True, labels=False, ticks=False, domain=False), title=None),
        color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave")
    ).properties(
        width=width, height=70
    )

    race_n_bar_early = race_n_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Early']}
    ).mark_bar(
        xOffset=-10,
        size=20, stroke=STROKE
    )
    race_n_bar_late = race_n_bar_base.transform_filter(
        {'field': 'wave', 'oneOf': ['Late']}
    ).mark_bar(
        xOffset=10,
        size=20, stroke=STROKE
    )
    race_n_bar = alt.layer(race_n_bar_early, race_n_bar_late)

    """
    /////////////////////
    LAYER-CHARTS FOR DIFF
    /////////////////////
    """
    diff = d.copy()
    diff.loc[diff.wave == 'Early', 'p_all'] = diff[diff.wave == 'Early'].p_all.apply(lambda x: -x)
    diff = diff.groupby(['group']).sum()
    diff = diff.reset_index()

    # VISUAL PARAMETERS
    xOffset = -7

    """
    AGE DIFF
    """
    age_diff = diff[diff.group.isin(ageGroups)]

    age_diff_chart = alt.Chart(
        age_diff
    ).mark_tick(
        color="black", size=40, stroke="white", thickness=5, xOffset=xOffset
    ).encode(
        x=alt.X("group:N", title=None, axis=None),
        y=alt.Y("p_all:Q")
    )

    """
    SEX DIFF
    """
    sex_diff = diff[diff.group.isin(sexGroups)]

    sex_diff_chart = alt.Chart(
        sex_diff
    ).mark_tick(
        color="black", size=40, stroke="white", thickness=5, xOffset=xOffset
    ).encode(
        x=alt.X("group:N", title=None, axis=None),
        y=alt.Y("p_all:Q")
    )

    """
    RACE DIFF
    """
    race_diff = diff[diff.group.isin(raceGroups)]

    race_diff_chart = alt.Chart(
        race_diff
    ).mark_tick(
        color="black", size=40, stroke="white", thickness=5, xOffset=xOffset
    ).encode(
        x=alt.X("group:N", title=None, axis=None),
        y=alt.Y("p_all:Q")
    )
    
    """
    ////////////////////////////////////
    ASSEMBLE
    ////////////////////////////////////
    """
    if race:
        final_chart = alt.hconcat(
            (age_p_bar + age_diff_chart), 
            (sex_p_bar + sex_diff_chart), 
            (race_p_bar + race_diff_chart)
        ).resolve_scale(y='shared', color='shared')
    
        nfinal_chart = alt.hconcat(age_n_bar, sex_n_bar, race_n_bar).resolve_scale(y='shared', color='shared')
    else:
        # Because we do not show race info for the countries other than USA
        final_chart = alt.hconcat(
            (age_p_bar + age_diff_chart), 
            (sex_p_bar + sex_diff_chart),
        ).resolve_scale(y='shared', color='shared')
    
        nfinal_chart = alt.hconcat(age_n_bar, sex_n_bar).resolve_scale(y='shared', color='shared')

    final_chart = (
        (final_chart & nfinal_chart).properties(
            title={
                "text": title,
                "dx": 80,
                "subtitle": get_visualization_subtitle(data_release='2021-01-25', with_num_sites=False), 
                "subtitleColor": "gray",
            }
        )
    )

    final_chart = apply_theme(
        final_chart,
        title_anchor='start',
        legend_orient='right'
    )
    return final_chart

In [ ]:
df2 = pd.read_csv(join("..", "data", "1.1.resurgence", "demographics", "demographic_stats.csv"))

all_chart = FUNC_DEMOGRAPHICS_BY_WAVE(df2, title="Demographics by Wave")
all_chart

In [ ]:
df.country.unique().tolist()

In [ ]:
USA_DF = df[df.country == "USA"]

usa_chart = FUNC_DEMOGRAPHICS_BY_WAVE(USA_DF, title="USA Demographics by Wave")
usa_chart

# USA_DF

In [ ]:
FRANCE_DF = df[df.country == "FRANCE"]

france_chart = FUNC_DEMOGRAPHICS_BY_WAVE(FRANCE_DF, title="France Demographics by Wave", race=False)
france_chart

# FRANCE_DF

In [ ]:
ITALY_DF = df[df.country == "ITALY"]

italy_chart = FUNC_DEMOGRAPHICS_BY_WAVE(ITALY_DF, title="Italy Demographics by Wave", race=False)
italy_chart

# ITALY_DF

In [ ]:
GERMANY_DF = df[df.country == "GERMANY"]

germany_chart = FUNC_DEMOGRAPHICS_BY_WAVE(GERMANY_DF, title="Germany Demographics by Wave", race=False)
germany_chart

# ITALY_DF

In [ ]:
SPAIN_DF = df[df.country == "SPAIN"]

spain_chart = FUNC_DEMOGRAPHICS_BY_WAVE(SPAIN_DF, title="Spain Demographics by Wave", race=False)
spain_chart

# ITALY_DF